# Create Delta Tables from Hive DDL

This notebook converts Hive/Impala DDL statements to Databricks Delta tables.

**Features:**
- Optional type optimization (STRING `_ts` columns → TIMESTAMP) - **disabled by default for safety**
- Managed Delta tables with auto-optimization
- Single file or batch processing
- Dry-run mode to preview DDL

**Prerequisites:**
- Hive DDL files uploaded to Volumes
- Appropriate permissions on target catalog/schema
- Active cluster (serverless or all-purpose)

## Setup

Import the table creation functions.

**Note:** Update the path below to point to where you've cloned the repo in your workspace.

In [ ]:
import sys
import os

# Get the directory where this notebook is located
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
notebook_dir = os.path.dirname(notebook_path)

# Add the schema migration tool to Python path (assumes notebook is in the tool directory)
sys.path.append(notebook_dir)

from create_delta_tables import create_tables_from_hive_ddl

print("✓ Imports successful")
print(f"Working directory: {notebook_dir}")

## Configuration

Set your target catalog, schema, and file paths

In [ ]:
# Target Databricks catalog and schema
TARGET_CATALOG = "your_catalog"  # UPDATE THIS
TARGET_SCHEMA = "your_schema"    # UPDATE THIS

print(f"Target location: {TARGET_CATALOG}.{TARGET_SCHEMA}")

## Option 1: Single File Processing

Convert a single Hive DDL file to Delta table.

**Note:** By default, column types are kept as-is (no automatic conversion).

In [ ]:
# Single file - UPDATE the path to your DDL file
SINGLE_FILE_PATH = "/Volumes/your_catalog/your_schema/your_path/table.sql"  # UPDATE THIS

result = create_tables_from_hive_ddl(
    input_file=SINGLE_FILE_PATH,
    catalog=TARGET_CATALOG,
    schema=TARGET_SCHEMA
    # optimize_types=False is the default (keeps original types, _ts as STRING)
    # If you want to convert _ts STRING columns to TIMESTAMP, set optimize_types=True
)

print(f"\n{'='*80}")
print("RESULT")
print(f"{'='*80}")
print(f"✓ Successfully created: {result['success_count']} table(s)")
print(f"✗ Failed: {result['fail_count']} table(s)")
print(f"Total processed: {result['total']} file(s)")

### Single File - Dry Run

Preview the DDL without creating tables

In [ ]:
# Dry run - just show what would be created
result = create_tables_from_hive_ddl(
    input_file=SINGLE_FILE_PATH,
    catalog=TARGET_CATALOG,
    schema=TARGET_SCHEMA,
    dry_run=True  # Only preview, don't create
)

## Option 2: Batch Processing

Process multiple DDL files from a directory

In [ ]:
# Batch processing - UPDATE the path to your directory containing .sql files
BATCH_DIRECTORY = "/Volumes/your_catalog/your_schema/your_path/hive_ddls/"  # UPDATE THIS

result = create_tables_from_hive_ddl(
    input_dir=BATCH_DIRECTORY,
    catalog=TARGET_CATALOG,
    schema=TARGET_SCHEMA
)

print(f"\n{'='*80}")
print("BATCH PROCESSING RESULT")
print(f"{'='*80}")
print(f"✓ Successfully created: {result['success_count']} table(s)")
print(f"✗ Failed: {result['fail_count']} table(s)")
print(f"Total processed: {result['total']} file(s)")

### Batch Processing - Dry Run

Preview all tables without creating them

In [ ]:
# Dry run for batch - see DDL for all files
result = create_tables_from_hive_ddl(
    input_dir=BATCH_DIRECTORY,
    catalog=TARGET_CATALOG,
    schema=TARGET_SCHEMA,
    dry_run=True  # Only preview, don't create
)

## Verify Created Tables

Check that tables were created successfully.

**Note:** Update the catalog and schema names in the SQL queries below.

In [ ]:
%sql
-- Show all tables in the target schema (UPDATE catalog and schema names)
SHOW TABLES IN your_catalog.your_schema;

In [ ]:
%sql
-- Describe a specific table (UPDATE catalog, schema, and table names)
DESCRIBE EXTENDED your_catalog.your_schema.your_table;

## Advanced Options

### WITH Type Optimization (Use with Caution)

⚠️ **WARNING:** Only enable if you're 100% certain your naming convention uses `_ts` suffix exclusively for timestamps.

This will convert STRING columns ending with `_ts` to TIMESTAMP.

**Risk:** Column names like `counts`, `status_ts`, `bytes` would be incorrectly converted!

In [ ]:
# Process WITH type optimization (risky!)
result = create_tables_from_hive_ddl(
    input_file=SINGLE_FILE_PATH,
    catalog=TARGET_CATALOG,
    schema=TARGET_SCHEMA,
    optimize_types=True  # ⚠️ Converts _ts STRING columns to TIMESTAMP
)

### Custom Catalog and Schema

Create tables in different catalog/schema combinations

In [ ]:
# Create in dev environment
result_dev = create_tables_from_hive_ddl(
    input_file=SINGLE_FILE_PATH,
    catalog="dev",
    schema="bronze"
)

# Create in prod environment
result_prod = create_tables_from_hive_ddl(
    input_file=SINGLE_FILE_PATH,
    catalog="prod",
    schema="bronze"
)

print(f"Dev: {result_dev['success_count']} tables created")
print(f"Prod: {result_prod['success_count']} tables created")

## Troubleshooting

### Check if schema exists

In [ ]:
%sql
-- UPDATE catalog name
SHOW SCHEMAS IN your_catalog;

### Check catalog permissions

In [ ]:
%sql
-- UPDATE catalog name
SHOW GRANTS ON CATALOG your_catalog;

### View table properties

In [ ]:
%sql
-- View Delta table properties (UPDATE catalog, schema, and table names)
SHOW TBLPROPERTIES your_catalog.your_schema.your_table;

## Notes

- Tables are created as **managed Delta tables** (no LOCATION clause)
- Partitioning is preserved from the Hive table
- Auto-optimization is enabled by default
- Tables are created **empty** (structure only, no data)
- **Column types are preserved by default** - no automatic conversion
- `_ts` columns remain as STRING unless you explicitly set `optimize_types=True`
- To load data, use separate `INSERT INTO` or data migration tools